**Creating Entities and relationships**

In [ ]:
# Defining the path to the 
file_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter17/Datasets/bank-full.csv'

In [ ]:
# Loading data using pandas
import pandas as pd
bankData = pd.read_csv(file_url,sep=";")
bankData.head()

In [ ]:
# Removing the target variable
Y = bankData.pop('y')

In [ ]:
# Creating the Ids for Demographic Entity
bankData['custID'] = bankData.index.values

bankData['custID'] = 'cust' + bankData['custID'].astype(str)

In [ ]:
# Creating AssetId
bankData['AssetId'] = 0
bankData.loc[bankData.housing == 'yes','AssetId']= 1

In [ ]:
# Creating LoanId
bankData['LoanId'] = 0
bankData.loc[bankData.loan == 'yes','LoanId']= 1

In [ ]:
# Creating Financial behaviour ID
bankData['FinbehId'] = 0
bankData.loc[bankData.default == 'yes','FinbehId']= 1

In [ ]:
# Importing necessary libraries
import featuretools as ft
import numpy as np

In [ ]:
# creating the entity set 'Bankentities'
Bankentities = ft.EntitySet(id = 'Bank')

In [ ]:
# Mapping a dataframe to the entityset to form the parent entity
Bankentities.entity_from_dataframe(entity_id = 'Demographic Data', dataframe = bankData, index = 'custID')

In [ ]:
# Mapping Assets and setting the relationship
Bankentities.normalize_entity(base_entity_id='Demographic Data', new_entity_id='Assets', index = 'AssetId', 
additional_variables = ['housing'])

In [ ]:
# Mapping Loans and Financial behavior entities
Bankentities.normalize_entity(base_entity_id='Demographic Data', new_entity_id='Liability', index = 'LoanId', 
additional_variables = ['loan'])

Bankentities.normalize_entity(base_entity_id='Demographic Data', new_entity_id='FinBehaviour', index = 'FinbehId', 
additional_variables = ['default'])

**Feature Engineering**

In [ ]:
# Creating feature sets using Deep Feature Synthesis
feature_set, feature_names = ft.dfs(entityset=Bankentities, 
target_entity = 'Demographic Data', 
max_depth = 2, 
verbose = 1, 
n_jobs = 1)

In [ ]:
# Reindexing the feature_set
feature_set = feature_set.reindex(index=bankData['custID'])
feature_set = feature_set.reset_index()

In [ ]:
# Verifying the shape of the features and original bank data
print(feature_set.shape)
print(bankData.shape)

In [ ]:
# Printing head of the feature set
feature_set.head()

In [ ]:
# Verifying the features for Assets.SUM(Demographic Data.balance)
bankData.groupby('AssetId')['balance'].agg('sum')

In [ ]:
# Printing the list of all features
feature_names

**Configuring the primitives**

In [ ]:
# Creating aggregation and transformation primitives
aggPrimitives=[
        'std', 'min', 'max', 'mean', 
         'last', 'count'
        
]
tranPrimitives=[
        'percentile', 
         'subtract', 'divide']

In [ ]:
# Defining the new set of features
feature_set, feature_names = ft.dfs(entityset=Bankentities, 
target_entity = 'Demographic Data',
agg_primitives=aggPrimitives,
trans_primitives=tranPrimitives, 
max_depth = 2, 
verbose = 1, 
n_jobs = 1)

In [ ]:
# Displaying the feature set 
feature_set.head()